# 模型训练及验证

上一节搭建了字符识别模型，接下来就是模型的训练及验证了，这里一般分为两个部分。第一部分是模型的训练，然后是测试集上测试其泛化性能。

## 1.模型训练

这里按数据集使用来划分，一般会将数据集拆分为三块，分别为训练数据train data,验证数据 val data 和测试数据test data。在训练模型时，模型参数的学习使用train data来学习。一般随着训练批次的增加，模型的train loss，即训练损失会越来越小，这时候模型对train数据集的特征学习的越来越好，但是会在没有训练的数据集上的误差会越来越大。这里的val data，是在训练过程中用来验证模型的泛化性能的数据集。如果模型评分在val data上变差，说明开始出现过拟合，此时需要停止训练。val data对我们模型的训练，调参具有很好的指导作用。

## 2.模型测试

训练完毕的模型，在之前完全没有参入模型训练的数据集上进行预测，得到评分，这个叫模型的测试阶段，因为test data模型完全没有接触过，所以可以测试模型的泛化性能。

# 数据集的准备

模型训练和验证之前，第一步是准确数据，这里需要用到pytorch的dataset 和dataloder类，分别构建数据读取及数据加载函数。

## 1. dataset 

这里使用pytorch的dataset类，继承其属性，构建我们自己的dataset

pythorch的dataset类在torch.utils.data.dataset的Dataset中，读取数据时，我们keys使用图片处理的方法进行数据增强，在torchvision中有数据增强的方法

In [1]:
from torch.utils.data.dataset import Dataset


In [4]:
import os, sys, glob, shutil, json
import cv2

from PIL import Image
import numpy as np

from tqdm import tqdm, tqdm_notebook

import torch
#torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable   

In [6]:
#定义自己的数据集类,从Dataset处继承
class MyDataset(Dataset):
    #定义初始化方法
    def __init__(self,img_path,img_label,transform=None):
        #三个参数，图片路径，图片标签和转化方式，转化方式与数据集扩展有关
        self.img_path=img_path
        self.img_label=img_label
        #注意我们可以先不进行数据增强，所以要加判断
        if transform !=None:
            self.transform=transform
        else:
            self.transform=None
    
    #构建数据集需要两个方法，一个是getitem,一个是len,这里我觉得getitem就是调用这个datasset后，会一个一个返回数据和标签的方法
    #len会返回数据集大小，这个getitem是按编号返回数据及标签。
    
    #这里的open是打开一个文件夹，注意，这里文件是一个一个读取的，img_path是一个list!所以有索引！！！，同理，label——path
    #也是一样.
    def __getitem__(self, index):
        #读取图片的方法可以换
        img = Image.open(self.img_path[index]).convert('RGB')
    #转换img
        if self.transform is not None:
            img = self.transform(img)
        
        # 设置最长的字符长度为5个,读取label,注意这里转换了一下数据类型
        lbl = np.array(self.img_label[index], dtype=np.int)
        #由于是固定长度，所以将不足5位的地方补足了
        lbl = list(lbl)  + (5 - len(lbl)) * [10]
        return img, torch.from_numpy(np.array(lbl[:5]))
   #这里再次注意img_path是一个列表
    def __len__(self):
        return len(self.img_path)

## 2.Dataloder

### 接下来就可以用DataLoder来加载定义好的Dataset了，需要我们指定好batch_size，即每次调用数据集的数量，是否打乱操作，和是否多线程。如果是window，则num_workers需要设置为0

In [8]:
#glob.glob读取结果是一个列表，类似于linnix的查找文件方式，可迭代对象
train_path = glob.glob('E:/Machine Learning/CV字符/mchar_train/mchar_train/*.png')
#列表排序
train_path.sort()
#json.load()是用来读取文件的，即，将文件打开然后就可以直接读取。示例如下
#with open("文件名") as f:
#    result=json.load(f)
train_json = json.load(open('E:/Machine Learning/CV字符/mchar_train.json'))
#json文件格式类似于字典，下面是一个试例
#{"000000.png": {"height": [219, 219], "label": [1, 9], "left": [246, 323], "top": [77, 81], "width": [81, 96]},
train_label = [train_json[x]['label'] for x in train_json]
print(len(train_path), len(train_label))

train_loader = torch.utils.data.DataLoader(
    MyDataset(train_path, train_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    transforms.RandomCrop((60, 120)),
                    transforms.ColorJitter(0.3, 0.3, 0.2),
                    transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=True, 
    num_workers=0,
)


30000 30000


### 同理，定义验证数据集

In [10]:
val_path = glob.glob('E:/Machine Learning/CV字符/mchar_val/mchar_val/*.png')
val_path.sort()
val_json = json.load(open('E:/Machine Learning/CV字符/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path), len(val_label))

val_loader = torch.utils.data.DataLoader(
    MyDataset(val_path, val_label,
                transforms.Compose([
                    #transforms.Resize((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
    num_workers=0,
)

10000 10000


## 3.构建模型

### 直接使用renset18作为特征提取模块，这里我其实不太明白什么叫特征提取模块，google一下

在本文中，主要是介绍提取中间层的特征，对于特征的提取，可以先把模型的结构输出，不同的模型结构是不一样的；下面拿resnet作为示例；由于pytorch模型很多用到nn.sequential,所以对各层的特征提取要自己去修改forward函数。这里可以看到，我们自定义了forward函数，由此可以选择在哪一层提取特征。

大概意思就是，在模型的某一神经层，提取其输出信息。具体哪一层需要在forward里修改。

In [ ]:
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()
        #这里选择了resnet，这里就需要对其网络结构比较熟悉，才能去相应层提取信息     
        model_conv = models.resnet18(pretrained=True)
        model_conv.avgpool = nn.AdaptiveAvgPool2d(1)
        model_conv = nn.Sequential(*list(model_conv.children())[:-1])
        self.cnn = model_conv
        
        self.fc1 = nn.Linear(512, 11)
        self.fc2 = nn.Linear(512, 11)
        self.fc3 = nn.Linear(512, 11)
        self.fc4 = nn.Linear(512, 11)
        self.fc5 = nn.Linear(512, 11)
    
    def forward(self, img):        
        feat = self.cnn(img)
        # print(feat.shape)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        return c1, c2, c3, c4, c5

### 4.训练，验证，测试模块 

下面定义好训练、验证和预测模块  

In [11]:
#后面的函数，我其实看不太懂了，要赶打卡进度，就先拷贝教材里的内容了
def train(train_loader, model, criterion, optimizer):
    # 切换模型为训练模式
    model.train()
    train_loss = []
    
    for i, (input, target) in enumerate(train_loader):
        if use_cuda:
            input = input.cuda()
            target = target.cuda()
            
        c0, c1, c2, c3, c4 = model(input)
        loss = criterion(c0, target[:, 0]) + \
                criterion(c1, target[:, 1]) + \
                criterion(c2, target[:, 2]) + \
                criterion(c3, target[:, 3]) + \
                criterion(c4, target[:, 4])
        
        # loss /= 6
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print(loss.item())
        
        train_loss.append(loss.item())
    return np.mean(train_loss)

def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if use_cuda:
                input = input.cuda()
                target = target.cuda()
            
            c0, c1, c2, c3, c4 = model(input)
            loss = criterion(c0, target[:, 0]) + \
                    criterion(c1, target[:, 1]) + \
                    criterion(c2, target[:, 2]) + \
                    criterion(c3, target[:, 3]) + \
                    criterion(c4, target[:, 4])
            # loss /= 6
            val_loss.append(loss.item())
    return np.mean(val_loss)

def predict(test_loader, model, tta=10):
    model.eval()
    test_pred_tta = None
    
    # TTA 次数
    for _ in range(tta):
        test_pred = []
    
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                
                c0, c1, c2, c3, c4 = model(input)
                output = np.concatenate([
                    c0.data.numpy(), 
                    c1.data.numpy(),
                    c2.data.numpy(), 
                    c3.data.numpy(),
                    c4.data.numpy()], axis=1)
                test_pred.append(output)
        
        test_pred = np.vstack(test_pred)
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

训练数据，记录loss值

In [ ]:
model = SVHN_Model1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.001)
best_loss = 1000.0

use_cuda = False
if use_cuda:
    model = model.cuda()

for epoch in range(2):
    train_loss = train(train_loader, model, criterion, optimizer, epoch)
    val_loss = validate(val_loader, model, criterion)
    
    val_label = [''.join(map(str, x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader, model, 1)
    val_predict_label = np.vstack([
        val_predict_label[:, :11].argmax(1),
        val_predict_label[:, 11:22].argmax(1),
        val_predict_label[:, 22:33].argmax(1),
        val_predict_label[:, 33:44].argmax(1),
        val_predict_label[:, 44:55].argmax(1),
    ]).T
    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str, x[x!=10])))
    
    val_char_acc = np.mean(np.array(val_label_pred) == np.array(val_label))
    
    print('Epoch: {0}, Train loss: {1} \t Val loss: {2}'.format(epoch, train_loss, val_loss))
    print(val_char_acc)
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), './model.pt')

后面就是测试数据集上，验证模型的泛化性能，生成预测结果

In [ ]:
test_path = glob.glob('../input/test_a/*.png')
test_path.sort()
test_label = [[1]] * len(test_path)
print(len(val_path), len(val_label))

test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    transforms.RandomCrop((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
    num_workers=0,
)

test_predict_label = predict(test_loader, model, 1)

test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))
    
import pandas as pd
df_submit = pd.read_csv('E:/Machine Learning/CV字符/test_A_sample_submit.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('renset18.csv', index=None)